<a href="https://colab.research.google.com/github/Zenrav/Text-to-image/blob/main/Text_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr
import requests
from PIL import Image
from io import BytesIO
import IPython.display as display
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", api_key = "your-api-key")
def generate_image(*prompts):
  valid_prompts = [str(prompt) for prompt in prompts if prompt]
  combined_words = ", ".join(valid_prompts)
  llm_response = llm.invoke(f"make a very short prompt containing the words:{combined_words}. Make sure to give good prompts and ensure all words are included. Give only 1 single prompt, nothing else")
  prompt_text = llm_response.content
  print(prompt_text)
  response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/generate/ultra",
    headers={
        "authorization": f"Bearer [auth-token]]",
        "accept": "image/*"
    },
    files={"none": ''},
    data={
        "prompt": prompt_text,
        "output_format": "webp",
    },
  )

  if response.status_code == 200:
      image = Image.open(BytesIO(response.content)).convert("RGB")
      display.display(image)  # Show image in notebook
  else:
      raise Exception(str(response.json()))
  return prompt_text, image
with gr.Blocks() as demo:
    number_input = gr.Number(label="Enter number of users", value=3, precision=0)
    @gr.render(inputs=[number_input])
    def create_boxes(number_input):
        user_words = []
        for i in range(number_input):
            with gr.Row(key=f'row-{i}'):
                user_prompt = gr.Textbox(
                    label=f"User {i+1}",
                    key=f"box-{i}",
                    interactive=True
                )
                user_words.append(user_prompt)
        with gr.Row():
            generate_btn = gr.Button("Generate Prompt and Image")
        output_text = gr.Textbox(label="Generated Prompt", interactive=False)
        output_image = gr.Image(label="Generated Image")
        generate_btn.click(
            fn=generate_image,
            inputs=user_words,
            outputs=[output_text, output_image]
        )

demo.launch(debug=True)
